<div align="center">
    <img width="300" src="https://2wz2rk1b7g6s3mm3mk3dj0lh-wpengine.netdna-ssl.com/wp-content/uploads/2018/08/kinetica_logo.svg" alt="Kinetica Logo"/>
</div>

# Python developer guide
The following guide provides step-by-step instructions to get started writing Python applications using Kinetica. This guide demonstrates only a small set of the available API. A detailed description of the complete interface is available under [Python API Reference](https://docs.kinetica.com/7.1/api/python/).

Go ahead and duplicate this notebook to run this on your own.




## Before you start
**Using deepnote**: You can duplicate this notebook to run it on your own.

**Install Kinetica**: You will need an instance of Kinetica that you can point to.

**Match the gpudb package version to your instance**: The version of gpudb package in your requirements.txt file needs to match the version of your instance of Kinetica. The package version should match upto the second level of a release. For instance, if you are running a 7.1.5.8 version of Kinetica, then the corresponding python package should be `gpudb==7.1.5.0`. 

To update the requirements, simply open up the requirements.txt file in the side panel and update it. Once it is updated, you will need to reinitialize your environment. To do this go to the environment tab on the left pane (the one below the folder icon) and run the init.ipynb notebook.

**If you run into issues**: Ask a question in our [community slack channel](https://join.slack.com/t/kinetica-community/shared_invite/zt-13ynqf304-bxuI_JKd9bW1BXny~Ze1QQ) and we will get back to you immediately. 

In [1]:
import gpudb
import collections
import json
import csv
import pandas as pd

## Connect to the database
Update the connection details below with those for your database server to connect to it.

In [ ]:
# Update the credentials to match those for your server
# options = gpudb.GPUdb.Options()
# options.username = "myusername"
# options.password = "mysecret"

# update the host to match your instance
# h_db = gpudb.GPUdb(
#   host=["<REPLACE WITH YOUR HOST URL"],
#   options = options
# )


# exec_result = h_db.show_system_status()['status_map']['system']
# print(exec_result)

In [2]:
options = gpudb.GPUdb.Options()
options.username = "kadmin"
options.password = "KineticaDemo123!"

h_db = gpudb.GPUdb(
  host=["https://mbrown-102521-euisfp7czn2h4.eastus2.cloudapp.azure.com/mbrown-102521-euisfp7czn2h4/gpudb-0"],
  options = options
)

#### For Developer edition
If you are using the developer edition of Kinetica the database is by default available on port 9191 on localhost (http://127.0.0.1:9191). If you are on an on-premise version of Kinetica, then go ahead and specify the connection URL for that server.

#### For Kinetica on the cloud
If you are using workbench (available on Azure and AWS at the moment) then you can use the connection details inside workbench to connect.

![](images/connect_workbench.png)

## Creating a Schema
All [tables](https://docs.kinetica.com/7.1/concepts/tables/) & [views](https://docs.kinetica.com/7.1/concepts/views/) are placed in a schema. Tables and views that are created without specifying a schema will be placed in the default schema of `ki_home`. We will use the schema `python_dev_guide` to manage all the tables and views for this guide. We can create a schema using [`create_schema()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.create_join_table).

In [6]:
h_db.create_schema("python_dev_guide", options={"no_error_if_exists": True})

{'schema_name': 'python_dev_guide',
 'info': {},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'create_schema_response',
  'response_time': 0.01281}}

## Creating a Type
Kinetica has a robust type inferencing system that can automatically infer the data types of different columns in a table that is being loaded into it. However, it is always best practice to explicitly define the types for a table before data is loaded. 

A [type](https://docs.kinetica.com/7.1/concepts/types/) is the column definition for the table. Each column comprises at least two values: a column name (always the first value) and a base type (always the second value). Any subsequent values are column properties. You can also use the [GPUdbRecordColumn](https://docs.kinetica.com/7.1/api/python/?source/gpudbrecordcolumn.html) and [GPUdbColumnProperty](https://docs.kinetica.com/7.1/api/python/?source/gpudbcolumnproperty.html) objects in addition to literal string values to help define a column. The following column type constants are available in [GPUdbRecordColumn](https://docs.kinetica.com/7.1/api/python/?source/gpudbrecordcolumn.html):

- _ColumnType.INT
- _ColumnType.LONG
- _ColumnType.FLOAT
- _ColumnType.DOUBLE
- _ColumnType.STRING
- _ColumnType.BYTES

> #### Important
> The order of the columns defines the order in which values must be inserted into the table; e.g., a `vendor_name` value must be inserted to the immediate right of the `vendor_id` value when using list-based insert methods--map-based insert methods will match the value with the correct column regardless of order.



## Table distribution
Kinetica is a distributed databases. Any table can be distributed across the Kinetica cluster using one of two schemes - [sharding](https://docs.kinetica.com/7.1/azure/concepts/tables/#sharding) or [replication](https://docs.kinetica.com/7.1/azure/concepts/tables/#replication). 

Sharding is a fundamental aspect of most distributed databases. It is performed by hashing a particular value for a record ([shard key](https://docs.kinetica.com/7.1/azure/concepts/tables/#shard-key)) to determine its location in the cluster. If a shard key is not specified but the table has a primary key, then the primary key becomes the default shard key. In the absence of both a shard key, a primary key and if the table is not specified as replicated, the table is randomly sharded and distributed across the cluster.

When done correctly, sharding has the benefit of dramatically increasing performance by distributing queries across the cluster. A limitation with sharding however, is that two sharded datasets can only be joined if they are sharded in the same way i.e. the records with the same values in both the tables are colocated.

Tables can also be replicated so that it available on every cluster node. The benefit of replication is that it allows data sets to be joined together when those data sets are not sharded on the columns being associated. Since making copies of the same table has storage costs, it is best and more common to replicate smaller tables. So for instance, in this guide, we replicate the vendors table.

## Creating Tables
After you've created a [type](https://docs.kinetica.com/7.1/concepts/types/) list and stored it in a variable, you can create a [table](https://docs.kinetica.com/7.1/concepts/tables/) using the [GPUdbTable](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html) interface. 

We will use three tables for this tuturial (the table distribution scheme is noted in brackets):

- A lookup table containing sample cab vendor information (replicated)
- A 1-to-1 relation table containing payment information for a subset of the cab-ride transactions (sharded on primary key `payment_id`)
- A base table containing a sample of NYC taxi cab ride data (sharded on `payment_id`)

In [8]:
# Create a type from a list of lists. Each list below is an individual
# column. Each column comprises at least two values: a column name (always
# the first value) and a base type (always the second value). Any
# subsequent values are column properties. The order of the columns defines
# the order in which values must be inserted into the table, e.g., a
# "vendor_name" value cannot be inserted before a "vendor_id" value
vendor_columns = [
    # column types and properties can be listed as strings
    ["vendor_id", "string", "char4", "primary_key"],
    ["vendor_name", "string", "char64"],
    ["phone", "string", "char16", "nullable"],
    ["email", "string", "char64", "nullable"],
    ["hq_street", "string", "char64"],
    # column types and properties can also be listed using the
    # GPUdbRecordColumn and GPUdbColumnProperty objects
    [
        "hq_city",
        gpudb.GPUdbRecordColumn._ColumnType.STRING,
        gpudb.GPUdbColumnProperty.CHAR8,
        gpudb.GPUdbColumnProperty.DICT
    ],
    [
        "hq_state",
        gpudb.GPUdbRecordColumn._ColumnType.STRING,
        gpudb.GPUdbColumnProperty.CHAR8,
        gpudb.GPUdbColumnProperty.DICT
    ],
    ["hq_zip", gpudb.GPUdbRecordColumn._ColumnType.INT],
    ["num_emps", gpudb.GPUdbRecordColumn._ColumnType.INT],
    ["num_cabs", gpudb.GPUdbRecordColumn._ColumnType.INT]
]

# Clear any existing table with the same name
h_db.clear_table(
    table_name="python_dev_guide.taxi_vendors", 
    options={"no_error_if_not_exists": True}
    )


# Create the table from the type and place it in a schema
try:
    table_vendor_obj = gpudb.GPUdbTable(
        _type=vendor_columns,
        name="python_dev_guide.taxi_vendors",
        options={"is_replicated": "true"},
        db=h_db
    )
    print("Vendor table successfully created")
except gpudb.GPUdbException as e:
    print("Vendor table creation failure: {}".format(str(e)))

Vendor table successfully created


To pass in table creation options, you can either specify a Python map, as above, or you can use the [GPUdbTableOptions](https://docs.kinetica.com/7.1/api/python/?source/gpudbtableoptions.html) object instead:

```python
creation_options = gpudb.GPUdbTableOptions.default().is_replicated(True)
```

Let's create a payments and taxi_trip_data table as well. These follow the same steps as above.

In [9]:
payment_columns = [
    ["payment_id", "long", "primary_key"],
    ["payment_type", "string", "char16", "nullable"],
    ["credit_type", "string", "char16", "nullable"],
    ["payment_timestamp", "long", "timestamp", "nullable"],
    ["fare_amount", "double", "nullable"],
    ["surcharge", "double", "nullable"],
    ["mta_tax", "double", "nullable"],
    ["tip_amount", "double", "nullable"],
    ["tolls_amount", "double", "nullable"],
    ["total_amount", "double", "nullable"]
]

# Clear any existing table with the same name
h_db.clear_table(
    table_name="python_dev_guide.payments", 
    options={"no_error_if_not_exists": True}
    )

# Create the table from the type and place it in a schema
try:
    table_payment_obj = gpudb.GPUdbTable(
        _type=payment_columns,
        name="python_dev_guide.payments",
        db=h_db
    )
    print("Payment table successfully created")
except gpudb.GPUdbException as e:
    print("Payment table creation failure: {}".format(str(e)))

Payment table successfully created


In [10]:
taxi_columns = [
    ["transaction_id", "long", "primary_key"],
    ["payment_id", "long", "primary_key", "shard_key"],
    ["vendor_id", "string", "char4"],
    ["pickup_datetime", "long", "timestamp"],
    ["dropoff_datetime", "long", "timestamp"],
    ["passenger_count", "int", "int8"],
    ["trip_distance", "float"],
    ["pickup_longitude", "float"],
    ["pickup_latitude", "float"],
    ["dropoff_longitude", "float"],
    ["dropoff_latitude", "float"]
]

# Clear any existing table with the same name (otherwise we won't be able
# to create the table)
h_db.clear_table(
    table_name="python_dev_guide.taxi_trips", 
    options={"no_error_if_not_exists": True}
    )

# Create the table from the type and place it in a schema
try:
    table_taxi_obj = gpudb.GPUdbTable(
        _type=taxi_columns,
        name="python_dev_guide.taxi_trips",
        db=h_db
    )
    print("Taxi table object successfully created")
except gpudb.GPUdbException as e:
    print("Taxi table object creation failure: {}".format(str(e)))

Taxi table object successfully created


## Load data into tables

Next let's explore adding some data into these tables using the [`insert_records()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.insert_records) method. We will first add a single row of data, followed by multiple records and then finally we will see how to load data from a CSV file.

> #### Note 
> Running the same insert multiple times will not add duplicate rows since each table has an associated primary key.

### Load single Key/Value records

In [11]:
# The number of rows in the payment table 
payment_table_size_before_insert = table_payment_obj.size()

# Create ordered dictionary for keys & values of record
payment_datum = collections.OrderedDict()
payment_datum["payment_id"] = 189
payment_datum["payment_type"] = "No Charge"
payment_datum["credit_type"] = None
payment_datum["payment_timestamp"] = None
payment_datum["fare_amount"] = 6.5
payment_datum["surcharge"] = 0
payment_datum["mta_tax"] = 0.6
payment_datum["tip_amount"] = 0
payment_datum["tolls_amount"] = 0
payment_datum["total_amount"] = 7.1

# Insert the record into the table
table_payment_obj.insert_records(payment_datum)
print("Number of records inserted into the Payment table: {}".format(
    table_payment_obj.size() - payment_table_size_before_insert
))

Number of records inserted into the Payment table: 1


### Multiple in-line records

The next two blocks of code add multiple records to the payment and vendor tables.

In [12]:
vendor_table_size_before_insert = table_vendor_obj.size()
# Create a list of in-line records. The order of the values must match the column order in the type
vendor_records = [
    ["VTS", "Vine Taxi Service", "9998880001", "admin@vtstaxi.com",
        "26 Summit St.", "Flushing", "NY", 11354, 450, 400],
    ["YCAB", "Yes Cab", "7895444321", None, "97 Edgemont St.", "Brooklyn",
        "NY", 11223, 445, 425],
    ["NYC", "New York City Cabs", None, "support@nyc-taxis.com",
        "9669 East Bayport St.", "Bronx", "NY", 10453, 505, 500],
    ["DDS", "Dependable Driver Service", None, None,
        "8554 North Homestead St.", "Bronx", "NY", 10472, 200, 124],
    ["CMT", "Crazy Manhattan Taxi", "9778896500",
        "admin@crazymanhattantaxi.com", "950 4th Road Suite 78", "Brooklyn",
        "NY", 11210, 500, 468],
    ["TNY", "Taxi New York", None, None, "725 Squaw Creek St.", "Bronx",
        "NY", 10458, 315, 305],
    ["NYMT", "New York Metro Taxi", None, None, "4 East Jennings St.",
        "Brooklyn", "NY", 11228, 166, 150],
    ["5BTC", "Five Boroughs Taxi Co.", "4566541278", "mgmt@5btc.com",
        "9128 Lantern Street", "Brooklyn", "NY", 11229, 193, 175]
]

# Insert the records into the Vendor table
table_vendor_obj.insert_records(vendor_records)

print("Number of records inserted into the Vendor table: {}".format(
    table_vendor_obj.size() - vendor_table_size_before_insert
))

Number of records inserted into the Vendor table: 8


In [14]:
payment_table_size_before_insert = table_payment_obj.size()
# Create another list of in-line records
payment_records = [
    [136, "Cash", None, 1428716521000, 4, 0.5, 0.5, 1, 0, 6.3],
    [148, "Cash", None, 1430124581000, 9.5, 0, 0.5, 1, 0, 11.3],
    [114, "Cash", None, 1428259673000, 5.5, 0, 0.5, 1.89, 0, 8.19],
    [180, "Cash", None, 1428965823000, 6.5, 0.5, 0.5, 1, 0, 8.8],
    [109, "Cash", None, 1428948513000, 22.5, 0.5, 0.5, 4.75, 0, 28.55],
    [132, "Cash", None, 1429472779000, 6.5, 0.5, 0.5, 1.55, 0, 9.35],
    [134, "Cash", None, 1429472668000, 33.5, 0.5, 0.5, 0, 0, 34.8],
    [176, "Cash", None, 1428403962000, 9, 0.5, 0.5, 2.06, 0, 12.36],
    [100, "Cash", None, None, 9, 0, 0.5, 2.9, 0, 12.7],
    [193, "Cash", None, None, 3.5, 1, 0.5, 1.59, 0, 6.89],
    [140, "Credit", "Visa", None, 28, 0, 0.5, 0, 0, 28.8],
    [161, "Credit", "Visa", None, 7, 0, 0.5, 0, 0, 7.8],
    [199, "Credit", "Visa", None, 6, 1, 0.5, 1, 0, 8.5],
    [159, "Credit", "Visa", 1428674487000, 7, 0, 0.5, 0, 0, 7.8],
    [156, "Credit", "MasterCard", 1428672753000, 12.5, 0.5, 0.5, 0, 0, 13.8],
    [198, "Credit", "MasterCard", 1429472636000, 9, 0, 0.5, 0, 0, 9.8],
    [107, "Credit", "MasterCard", 1428717377000, 5, 0.5, 0.5, 0, 0, 6.3],
    [166, "Credit", "American Express", 1428808723000, 17.5, 0, 0.5, 0, 0, 18.3],
    [187, "Credit", "American Express", 1428670181000, 14, 0, 0.5, 0, 0, 14.8],
    [125, "Credit", "Discover", 1429869673000, 8.5, 0.5, 0.5, 0, 0, 9.8],
    [119, None, None, 1430431471000, 9.5, 0, 0.5, 0, 0, 10.3],
    [150, None, None, 1430432447000, 7.5, 0, 0.5, 0, 0, 8.3],
    [170, "No Charge", None, 1430431502000, 28.6, 0, 0.5, 0, 0, 28.6],
    [123, "No Charge", None, 1430136649000, 20, 0.5, 0.5, 0, 0, 21.3],
    [181, None, None, 1430135461000, 6.5, 0.5, 0.5, 0, 0, 7.8]
]

# Insert the records into the Payment table
for record in payment_records:
    table_payment_obj.insert_records(record)

print("Number of records inserted into the Payment table: {}".format(
    table_payment_obj.size() - payment_table_size_before_insert
))

Number of records inserted into the Payment table: 0


### Load from a CSV file

Let's upload all the data from the taxi_trip_data.csv file to the table_taxi

In [15]:
with open('data/taxi_trip_data.csv') as csvfile:
    taxi_data = csv.reader(csvfile, delimiter=',')
    header = []
    header = next(taxi_data)
    taxi_records = []
    for record in taxi_data:
        record_data = []
        record_data.append(int(record[0]))
        record_data.append(int(record[1]))
        record_data.append(record[2])
        record_data.append(int(record[3]))
        record_data.append(int(record[4]))
        record_data.append(int(record[5]))
        record_data.append(float(record[6]))
        record_data.append(float(record[7]))
        record_data.append(float(record[8]))
        record_data.append(float(record[9]))
        record_data.append(float(record[10]))
        taxi_records.append(record_data)
    table_taxi_obj.insert_records(taxi_records)

    

### Load from data source

Add this as well to showcase newer capabilities. Maybe a streaming data source?

## Retrieving records from Kinetica
Once the [table](https://docs.kinetica.com/7.1/concepts/tables/) is populated with data, the data can be retrieved using binary, JSON or geoJSON 
encoding. Binary encoding is more efficient than JSON encoding.

The [`get_records()`](https://docs.kinetica.com/7.1/api/python/frame/source/gpudb.html?highlight=get%20records#gpudb.GPUdb.get_records) method comes in a few different flavors. You have the option to retrieve an entire table or `get_records_by_column()` additionally, you can choose the `get_records_and_decode()` option to retrieve the records as an ordered dictionary. The example below shows, the latter. 

The limit and offset options can be used to retrieve smaller blocks of data when you have large tables.

In [ ]:
# Retrieve the record from table in Kinetica
records = h_db.get_records_and_decode(
    table_name="python_dev_guide.taxi_trips", 
    limit = 5, 
    encoding='json')['records']

# Convert to a pandas data frame and print
df = pd.DataFrame(records)
display(df)

,transaction_id,payment_id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,199751611,166,YCAB,1428808559000,1428808723000,2,0.50,-73.986794,40.760735,-73.982506,40.767475
1,191386113,187,YCAB,1428669361000,1428670181000,1,1.30,-73.970818,40.759006,-73.983658,40.752861
2,106041671,125,NYC,1429869346000,1429869673000,1,0.79,-73.984428,40.769939,-73.992729,40.762539
3,177106037,119,YCAB,1430431112000,1430431471000,1,1.20,-73.988785,40.744782,-74.000481,40.734825
4,169677687,150,YCAB,1430431112000,1430432447000,1,6.50,-73.975449,40.765274,-74.009109,40.704418


## Updating and removing records
Using any [GPUdbTable](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html) object, you can update records via the update_records method.

In [ ]:
# Update the e-mail of, and add two employees and one cab to, the DDS vendor
table_vendor_obj.update_records(
    expressions=["vendor_id = 'DDS'"],
    new_values_maps={
        "email": "'management@ddstaxico.com'",
        "num_emps": "num_emps + 2",
        "num_cabs": "num_cabs + 1"
    },
    options={"use_expressions_in_new_values_maps":"true"}
)

{'count_updated': 1,
 'counts_updated': [1],
 'count_inserted': 0,
 'counts_inserted': [0],
 'info': {},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'update_records_response',
  'response_time': 0.01551}}

You can delete records using the [`delete_records()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.delete_records) method.

In [ ]:
# Delete payment 189
pre_delete = table_payment_obj.size()
print("Records in the payment table (before delete): {}".format(pre_delete))
delete_expr = ["payment_id = 189"]
print("Deleting record where " + delete_expr[0])
table_payment_obj.delete_records(expressions=delete_expr)
post_delete = table_payment_obj.size()
print("Records in the payment table (after delete): {}".format(post_delete))

Records in the payment table (before delete): 26
Deleting record where payment_id = 189
Records in the payment table (after delete): 25


## Alter Table
Some properties can be altered or added after [table](https://docs.kinetica.com/7.1/concepts/tables/) creation, including [indexes](https://docs.kinetica.com/7.1/concepts/indexes/), [dictionary encoding](https://docs.kinetica.com/7.1/concepts/dictionary_encoding/), and [compression](https://docs.kinetica.com/7.1/concepts/compression/).

### Indexes
The example below uses the [`alter_table()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.alter_table) method to create column indexes (also known as attribute index) on the `fare_amonunt` and `passenger_count` columns in the payment table.



In [ ]:
# Add column indexes on:
#   - payment table, fare_amount (for query-chaining filter example)
#   - taxi table, passenger_count (for filter-by-range example) */
table_payment_obj.alter_table(action="create_index", value="fare_amount")
table_taxi_obj.alter_table(action="create_index", value="passenger_count")

{'table_name': 'python_dev_guide.taxi_trips',
 'action': 'create_index',
 'value': 'passenger_count',
 'type_id': '',
 'type_definition': '',
 'properties': {},
 'label': '',
 'info': {},
 'status_info': {'status': 'OK',
  'message': '',
  'data_type': 'alter_table_response',
  'response_time': 0.01148}}

### Compression
Applying column compression works similarly: using the [`alter_table()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.alter_table) method, but with a `set_column_compression` action paired with a column name and compression type option.

In [ ]:
# Apply the snappy compression algorithm to the pickup and dropoff datetime columns
snappy_compression = {"compression_type": "snappy"}
table_taxi_obj.alter_table(
    action="set_column_compression",
    value="pickup_datetime",
    options=snappy_compression
)

dropoff_compress_resp = table_taxi_obj.alter_table(
    action="set_column_compression",
    value="dropoff_datetime",
    options=snappy_compression
)

>#### Important
> Column compression is applied at a fixed interval, so be sure to verify later that the compression has been added. Column usage should decrease by roughly 23% (~1989 bytes)


### Dictionary Encoding
Applying dictionary encoding via [`alter_table()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.alter_table) involves adding a new property to a column.

In [ ]:
# Apply dictionary encoding to the vendor_id column
table_taxi_obj.alter_table(
    action="change_column",
    value="vendor_id",
    options={"column_properties": "char4,dict"}
)

{'table_name': 'python_dev_guide.taxi_trips',
 'action': 'change_column',
 'value': 'vendor_id',
 'type_id': '6085055925073034879',
 'type_definition': '{"type":"record","name":"type_name","fields":[{"name":"transaction_id","type":"long"},{"name":"payment_id","type":"long"},{"name":"vendor_id","type":"string"},{"name":"pickup_datetime","type":"long"},{"name":"dropoff_datetime","type":"long"},{"name":"passenger_count","type":"int"},{"name":"trip_distance","type":"float"},{"name":"pickup_longitude","type":"float"},{"name":"pickup_latitude","type":"float"},{"name":"dropoff_longitude","type":"float"},{"name":"dropoff_latitude","type":"float"}]}',
 'properties': {'dropoff_datetime': ['data', 'timestamp'],
  'dropoff_latitude': ['data'],
  'dropoff_longitude': ['data'],
  'passenger_count': ['data', 'int8'],
  'payment_id': ['data', 'primary_key', 'shard_key'],
  'pickup_datetime': ['data', 'timestamp'],
  'pickup_latitude': ['data'],
  'pickup_longitude': ['data'],
  'transaction_id': ['dat

## Filters
Filters are an easy way to reduce larger [tables](https://docs.kinetica.com/7.1/concepts/tables/) into more concise [views](https://docs.kinetica.com/7.1/concepts/views/) using [expressions](https://docs.kinetica.com/7.1/concepts/expressions/).

### Filter Example 1
In the example below Kinetica creates a view with a random name.

In [ ]:
# Select all payments with no corresponding payment type; allow Kinetica 
# to assign a random name to the view
f1_count = table_payment_obj.filter(expression="IS_NULL(payment_type)").size()

print("Number of null payments: {}".format(f1_count))

Number of null payments: 3


### Filter example 2

In [ ]:
# Using GPUdbTable query chaining, filter null payment type records with a
# fare amount greater than 8
f2_count = table_payment_obj.filter(
    view_name="python_dev_guide.view_example1",
    expression="IS_NULL(payment_type)"
    ).filter(
    view_name="python_dev_guide.view_example2",
    expression="fare_amount > 8"
    ).size()

print("Number of null payments with a fare amount greater than $8.00 " \
    "(with query chaining): {}".format(f2_count))

Number of null payments with a fare amount greater than $8.00 (with query chaining): 1


### Filter example 3

In [ ]:
# Filter by list where vendor ID is either NYC or YCAB
f3_count = table_taxi_obj.filter_by_list(
    view_name="python_dev_guide.view_example3",
    column_values_map={"vendor_id": ["NYC", "YCAB"]}
).size()

print("Number of records where vendor_id is either NYC or YCAB: {}".format(f3_count))

Number of records where vendor_id is either NYC or YCAB: 498


### Filter example 4

In [ ]:
# Filter by range trip with passenger count between 1 and 3
f4_count = table_taxi_obj.filter_by_range(
    view_name="python_dev_guide.view_example4",
    column_name="passenger_count",
    lower_bound=1,
    upper_bound=3
).size()

print("Number of trips with a passenger count between 1 and 3: {}".format(f4_count))

Number of trips with a passenger count between 1 and 3: 972


## Aggregates
Kinetica supports various aggregate and group-by queries, which group and aggregate your data to return counts and useful statistics.

### Aggregate Example 1

In [ ]:
# Aggregate count, min, mean, and max on the trip distance
a1_resp = table_taxi_obj.aggregate_statistics(
    column_name="trip_distance",
    stats="count,min,max,mean"
)

print("Statistics of values in the trip_distance column:")
print("\tCount: {count:.0f}\n\tMin: {min:4.2f}\n\tMean: {mean:4.2f}\n\t" \
    "Max: {max:4.2f}\n".format(**a1_resp["stats"]))

Statistics of values in the trip_distance column:
	Count: 1081
	Min: 0.00
	Mean: 2.86
	Max: 21.88



### Aggregate Example 2

In [ ]:
# Find unique taxi vendor IDs
a2_resp = table_taxi_obj.aggregate_unique(
    column_name="vendor_id",
    offset=0,
    limit=gpudb.GPUdb.END_OF_SET,
    encoding="json"
)["data"]["vendor_id"]

print("Unique vendor IDs in the Taxi table:")
for vendor in a2_resp:
    print("\t* {}".format(vendor))

Unique vendor IDs in the Taxi table:
	* CMT
	* DDS
	* LYFT
	* NYC
	* UBER
	* VTS
	* YCAB


### Aggregate Example 3

In [ ]:
# Find number of trips per vendor
a3_resp = table_taxi_obj.aggregate_group_by(
    column_names=["vendor_id", "COUNT(vendor_id)"],
    offset=0,
    limit=gpudb.GPUdb.END_OF_SET,
    options={"sort_by": "key"}
)["data"]

print("Number of trips per vendor:")
for vendor in zip(a3_resp["vendor_id"], a3_resp["COUNT(vendor_id)"]):
    print("\t{:<10s}{:4d}".format(vendor[0] + ":", vendor[1]))

Number of trips per vendor:
	CMT:       242
	DDS:       106
	LYFT:        3
	NYC:       262
	UBER:        3
	VTS:       229
	YCAB:      236


### Aggregate Example 4

In [ ]:
# Create a histogram for the different groups of passenger counts
a4_resp = table_taxi_obj.aggregate_histogram(
    column_name="passenger_count",
    start=1,
    end=6,
    interval=1
)["counts"]

print("Passenger count groups by size:")
print("{:<10s} {:<11s}".format("Passengers", "Total Trips"))
print("{:=<10s} {:=<11s}".format("", ""))

for histo_group in zip([1, 2, 3, 4, '>5'], a4_resp):
    print("{:>10} {:11.0f}".format(*histo_group))

Passenger count groups by size:
Passengers Total Trips
========== ===========
         1         775
         2         145
         3          52
         4          22
        >5          87


## Joins
[Joins](https://docs.kinetica.com/7.1/concepts/joins/) allow you to link multiple [tables](https://docs.kinetica.com/7.1/concepts/tables/) together, along their relations, retrieving associated information from any or all of them. Tables can only be joined if they're [sharded](https://docs.kinetica.com/7.1/concepts/tables/#sharded) similarly or [replicated](https://docs.kinetica.com/7.1/concepts/tables/#replicated). Note that the [`create_join_table()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.create_join_table) method is static and is not called with a corresponding [GPUdbTable](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html) object.

### Inner join
An inner join returns only records that have matching values in both tables. The join below is done using the common shard key `payment_id`.

In [ ]:
# Retrieve payment information for rides having more than three passengers
gpudb.GPUdbTable.create_join_table(
    join_table_name="python_dev_guide.join_table_inner",
    table_names=[
        "python_dev_guide.taxi_trips as t",
        "python_dev_guide.payments as p"
    ],
    column_names=[
        "t.payment_id",
        "payment_type",
        "total_amount",
        "passenger_count",
        "vendor_id",
        "trip_distance"
    ],
    expressions=[
        "t.payment_id = p.payment_id",
        "passenger_count > 3"
    ],
    db=h_db
)

### Left join
A left join returns all of the records an inner join does, but additionally, for each record in the table on the left side of the join that has no match along the relation to a record in the table on the right side of the join, a corresponding record will be returned with "left-side" columns populated with the "left-side" record data and the "right-side" columns populated with nulls.

In [ ]:
# Retrieve cab ride transactions and the full name of the associated vendor
# (if available--blank if vendor name is unknown) for transactions with
# associated payment data, sorting by increasing values of transaction ID.
gpudb.GPUdbTable.create_join_table(
    join_table_name="python_dev_guide.join_table_left",
    table_names=[
        "python_dev_guide.taxi_trips as t",
        "python_dev_guide.taxi_vendors as v"
    ],
    column_names=[
        "transaction_id",
        "pickup_datetime",
        "trip_distance",
        "t.vendor_id",
        "vendor_name"
    ],
    expressions=[
        "LEFT JOIN t, v ON (t.vendor_id = v.vendor_id)",
        "payment_id <> 0"
    ],
    db=h_db
)

### Outer join
Full outer joins require both tables to be replicated or joined on their shard keys. Set merges that perform deduplication of records, like [Union Distinct](https://docs.kinetica.com/7.1/concepts/unions/), [Intersect](https://docs.kinetica.com/7.1/concepts/intersect/), and [Except](https://docs.kinetica.com/7.1/concepts/except/) also need to use replicated tables to ensure the correct results, so a replicated version of the taxi (taxi_trip_data_replicated) table and a corresponding table object are created at this point in the tutorial.

In [ ]:
# Clear any existing table with the same name
h_db.clear_table(
    table_name="python_dev_guide.table_taxi_replicated", 
    options={"no_error_if_not_exists": True}
    )


# Copy the records from the taxi trip table to a replicated taxi trip 
# table using merge_records
gpudb.GPUdbTable.merge_records(
    table_name="python_dev_guide.table_taxi_replicated", 
    source_table_names=["python_dev_guide.taxi_trips"],
    field_maps=[{
        "transaction_id": "transaction_id",
        "payment_id": "payment_id",
        "vendor_id": "vendor_id",
        "pickup_datetime": "pickup_datetime",
        "dropoff_datetime": "dropoff_datetime",
        "passenger_count": "passenger_count",
        "trip_distance": "trip_distance",
        "pickup_longitude": "pickup_longitude",
        "pickup_latitude": "pickup_latitude",
        "dropoff_longitude": "dropoff_longitude",
        "dropoff_latitude": "dropoff_latitude"
    }],
    options={"is_replicated": "true"},
    db=h_db
)

table_taxi_replicated_obj = gpudb.GPUdbTable(
        _type=None,
        name="python_dev_guide.table_taxi_replicated",
        db=h_db
        )


A full outer join returns all of the records a left join does, but additionally, for each record in the table on the right side of the join that has no match along the relation to a record in the table on the left side of the join, a corresponding record will be returned with "right-side" columns populated with the "right-side" record data and the "left-side" columns populated with nulls.

In [ ]:
# Retrieve the vendor IDs of known vendors with no recorded cab ride
# transactions, as well as the vendor ID and number of transactions for
# unknown vendors with recorded cab ride transactions
gpudb.GPUdbTable.create_join_table(
    join_table_name="python_dev_guide.join_table_outer",
    table_names=[
        "python_dev_guide.table_taxi_replicated as t",
        "python_dev_guide.taxi_vendors as v"
    ],
    column_names=[
        "t.vendor_id as vendor_id",
        "v.vendor_id as vendor_id_1"
    ],
    expressions=["FULL_OUTER JOIN t,v ON ((v.vendor_id = t.vendor_id))"],
    db=h_db
)

## Projections
You can create [projections](https://docs.kinetica.com/7.1/concepts/projections/) using the [`create_projection()`](https://docs.kinetica.com/7.1/api/python/?source/gpudbtable.html#gpudb.GPUdbTable.create_projection) method.

In [ ]:
# Create a projection containing all payments by credit card
table_payment_obj.create_projection(
    projection_name="python_dev_guide.projection_example1",
    column_names=[
        "payment_id",
        "payment_type",
        "credit_type",
        "payment_timestamp",
        "fare_amount",
        "surcharge",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "total_amount"
    ],
    options={"expression": "payment_type = 'Credit'"}
)

To persist a projection:



In [ ]:
# Create a persisted table with cab ride transactions greater than 5 miles
# whose trip started during lunch hours
table_taxi_obj.create_projection(
    projection_name="python_dev_guide.projection_example2",
    column_names=[
        "HOUR(pickup_datetime) as hour_of_day",
        "vendor_id",
        "passenger_count",
        "trip_distance"
    ],
    options={
        "expression":
            "(HOUR(pickup_datetime) >= 11) AND "
            "(HOUR(pickup_datetime) <= 14) AND "
            "(trip_distance > 5)",
        "persist": "true"
    }
)

## Union, Intersect, and Except
[Union](https://docs.kinetica.com/7.1/concepts/unions/) can be used to combine homogeneous data sets into one larger data set. Union & Union Distinct will both combine data sets but only retain the records that are unique across the chosen columns, removing all duplicates. Union All will combine data sets, retaining all records from the source data sets.

In [ ]:
# Calculate the average number of passengers, as well as the shortest,
# average, and longest trips for all trips in
# each of the two time periods--from April 1st through the 15th, 2015 and
# from April 16th through the 23rd, 2015--and return those two sets of
# statistics in a single result set.
table_taxi_obj.aggregate_group_by(
    column_names=[
        "AVG(passenger_count) as avg_pass_count",
        "AVG(trip_distance) as avg_trip_dist",
        "MIN(trip_distance) as min_trip_dist",
        "MAX(trip_distance) as max_trip_dist"
    ],
    offset=0,
    limit=gpudb.GPUdb.END_OF_SET,
    options={
        "expression":
            "(pickup_datetime >= '2015-04-01') AND "
            "(pickup_datetime <= '2015-04-15 23:59:59.999')",
        "result_table": "python_dev_guide.agg_grpby_union_all_src1"
    }
)
table_taxi_obj.aggregate_group_by(
    column_names=[
        "AVG(passenger_count) as avg_pass_count",
        "AVG(trip_distance) as avg_trip_dist",
        "MIN(trip_distance) as min_trip_dist",
        "MAX(trip_distance) as max_trip_dist"
    ],
    offset=0,
    limit=gpudb.GPUdb.END_OF_SET,
    options={
        "expression":
            "(pickup_datetime >= '2015-04-16') AND "
            "(pickup_datetime <= '2015-04-23 23:59:59.999')",
        "result_table": "python_dev_guide.agg_grpby_union_all_src2"
    }
)
gpudb.GPUdbTable.create_union(
    table_name="python_dev_guide.union_all_table",
    table_names=[
        "python_dev_guide.agg_grpby_union_all_src1",
        "python_dev_guide.agg_grpby_union_all_src2"
    ],
    input_column_names=[
        ["'2015-04-01 - 2014-04-15'", "avg_pass_count",
            "avg_trip_dist", "min_trip_dist", "max_trip_dist"],
        ["'2015-04-16 - 2015-04-23'", "avg_pass_count",
            "avg_trip_dist", "min_trip_dist", "max_trip_dist"]
    ],
    output_column_names=[
        "pickup_window_range",
        "avg_pass_count",
        "avg_trip",
        "min_trip",
        "max_trip"
    ],
    options={"mode": "union_all"},
    db=h_db
)

[Intersect](https://docs.kinetica.com/7.1/concepts/intersect/) will combine data sets but only include the records found in both data sets, removing duplicate result records.

In [ ]:
# Retrieve locations (as lat/lon pairs) that were both pick-up and
# drop-off points
gpudb.GPUdbTable.create_union(
    table_name="python_dev_guide.union_intersect_table",
    table_names=[
        "python_dev_guide.table_taxi_replicated",
        "python_dev_guide.table_taxi_replicated"
    ],
    input_column_names=[
        ["pickup_latitude", "pickup_longitude"],
        ["dropoff_latitude", "dropoff_longitude"]
    ],
    output_column_names=["latitude", "longitude"],
    options={"mode": "intersect"},
    db=h_db
)

[Except](https://docs.kinetica.com/7.1/concepts/except/) will return records that appear in the first data set but not the second data set. Note that the data sets on each side of the Except will have duplicates removed first, and then the set subtraction will be processed.

In [ ]:
# Show vendors that operate before noon, but not after noon: retrieve the
# unique list of IDs of vendors who provided cab rides between midnight
# and noon, and remove from that list the IDs of any vendors who provided
# cab rides between noon and midnight
table_taxi_replicated_obj.create_projection(
    projection_name="python_dev_guide.projection_except_src1",
    column_names=["vendor_id"],
    options={
        "expression":
            "(HOUR(pickup_datetime) >= 0) AND "
            "(HOUR(pickup_datetime) <= 11)"
    }
)
table_taxi_replicated_obj.create_projection(
    projection_name="python_dev_guide.projection_except_src2",
    column_names=["vendor_id"],
    options={
        "expression":
            "(HOUR(pickup_datetime) >= 12) AND "
            "(HOUR(pickup_datetime) <= 23)"
    }
)
gpudb.GPUdbTable.create_union(
    table_name="python_dev_guide.union_except_table",
    table_names=[
        "python_dev_guide.projection_except_src1",
        "python_dev_guide.projection_except_src2"
    ],
    input_column_names=[
        ["vendor_id"],
        ["vendor_id"]
    ],
    output_column_names=["vendor_id"],
    options={"mode": "except"},
    db=h_db
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3504dc9a-cf94-4c79-ba37-d319423beb20' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>